In [1]:
import os

cwd = os.getcwd()

In [2]:
import pandas as pd


df = pd.read_csv(cwd + '/data/processed/posts_train_test_clean.csv')

In [3]:
df.head()

,Unnamed: 0,Post Link,Title,Body,Tags,CreationDate,Answer Date,AcceptedAnswerId,id,score,complete_text,normalized_text
0,69943,16835963,Sidekiq not processing queue,"<p>What possible reasons can <a href=""https://...",<ruby-on-rails-3><redis><sidekiq>,2013-05-30 12:19:00,2013-05-30 12:53:32,16836794,16836794,71,sidekiq not processing queue what possible rea...,sidekiq process queue possibl reason sidekiq p...
1,79655,31667160,Running docker container : iptables: No chain/...,<p>I'm trying to run a container but I get the...,<docker><port><iptables>,2015-07-28 04:03:53,2015-07-28 04:14:04,31667244,31667244,65,running docker container iptables no chain tar...,run docker contain iptabl chain target match n...
2,97858,3299648,Python Compilation/Interpretation Process,<p>I'm trying to understand the python compile...,<python><compiler-construction><interpreter><p...,2010-07-21 13:21:47,2010-07-21 13:28:54,3299724,3299724,53,python compilation interpretation process i m ...,python compil interpret process tri understand...
3,14078,3664272,Is std::vector so much slower than plain arrays?,<p>I've always thought it's the general wisdom...,<c++><arrays><performance><stl><vector>,2010-09-08 02:38:41,2010-09-08 02:57:40,3664349,3664349,239,is std vector so much slower than plain arrays...,std vector much slower plain array alway thoug...
4,46299,13342123,How to get relative path of a file in visual s...,<p>I am trying to get the path of an image fil...,<c#><visual-studio>,2012-11-12 10:33:32,2013-10-11 10:57:06,19316470,19316470,100,how to get relative path of a file in visual s...,get rel path file visual studio tri get path i...


## Criando os targets

In [4]:
def split_tags(tags):
    tags = tags[1:-1]
    tags = tags.split("><")
    
    return tags

In [5]:
df['targets'] = df.Tags.apply(lambda x: split_tags(x))

In [6]:
df['targets']

0                        [ruby-on-rails-3, redis, sidekiq]
1                                 [docker, port, iptables]
2        [python, compiler-construction, interpreter, p...
3                  [c++, arrays, performance, stl, vector]
4                                      [c#, visual-studio]
                               ...                        
73917                           [javascript, jquery, html]
73918                      [android-studio, jetbrains-ide]
73919                                   [html, html-table]
73920                     [javascript, arrays, typescript]
73921                     [http, dart, localhost, flutter]
Name: targets, Length: 73922, dtype: object

## 100 Tags mais usadas

In [7]:
def calculateTagsEntries(df):
    count = {}
    
    for tags in df['targets']:
        for tag in tags:
            if(tag not in count.keys()):
                dataTag = {
                    'entries': 1,
                }
                count[tag] = dataTag
            else:
                count[tag]['entries'] += 1
    return count

def sortTags(count):
    return dict(sorted(count.items(), key=lambda item: -item[1]['entries']))

In [8]:
classes = []

def createClassesColumn(df):
    for documentTags in df['targets']:
        wasInDocumentTags = False

        for documentTag in documentTags:
            if(documentTag in MostUsedTags):
                wasInDocumentTags = True
                classes.append(documentTag)
                break
        if(not wasInDocumentTags):
            classes.append('unknown-tag')
    df['class'] = classes    

In [9]:
MostUsedTags = list(sortTags(calculateTagsEntries(df)).keys())[:100]
createClassesColumn(df)

In [10]:
 y = df['class']

In [11]:
y

0           unknown-tag
1                docker
2                python
3                   c++
4                    c#
              ...      
73917        javascript
73918    android-studio
73919              html
73920        javascript
73921              http
Name: class, Length: 73922, dtype: object

In [12]:
targets_set = set(MostUsedTags + ['unknown-tag'])

In [13]:
len(targets_set)

101

In [14]:
targets_set

{'.net',
 'ajax',
 'algorithm',
 'amazon-web-services',
 'android',
 'android-studio',
 'angular',
 'angularjs',
 'arrays',
 'asp.net',
 'asp.net-mvc',
 'bash',
 'c',
 'c#',
 'c++',
 'c++11',
 'class',
 'cocoa-touch',
 'command-line',
 'css',
 'dart',
 'database',
 'dataframe',
 'date',
 'datetime',
 'debugging',
 'dictionary',
 'django',
 'docker',
 'eclipse',
 'entity-framework',
 'exception',
 'file',
 'flutter',
 'function',
 'git',
 'github',
 'go',
 'google-chrome',
 'gradle',
 'html',
 'http',
 'image',
 'intellij-idea',
 'ios',
 'iphone',
 'java',
 'java-8',
 'javascript',
 'jquery',
 'json',
 'laravel',
 'linq',
 'linux',
 'list',
 'macos',
 'matplotlib',
 'maven',
 'mongodb',
 'multithreading',
 'mysql',
 'node.js',
 'npm',
 'numpy',
 'objective-c',
 'oop',
 'pandas',
 'performance',
 'php',
 'postgresql',
 'python',
 'python-3.x',
 'r',
 'reactjs',
 'regex',
 'rest',
 'ruby',
 'ruby-on-rails',
 'scala',
 'shell',
 'sorting',
 'spring',
 'sql',
 'sql-server',
 'string',
 'swi

In [15]:
def buildMulticlassTestArrays(targets_set, tags):
    entries = []
    for tagList in tags:
        entry = []
        for target in targets_set:
            if(target in tagList):
                entry.append(1)
            else:
                entry.append(0)
        entries.append(entry)
    return entries

In [16]:
words_post = df.normalized_text.apply(lambda x: str(x))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(words_post)
X = vectorizer.transform(words_post)

In [17]:
import gc

gc.collect()

0

In [18]:
from sklearn.model_selection import train_test_split
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#y_test = y_test.apply(lambda x: [x])

In [19]:
from sklearn.ensemble import RandomForestClassifier
# rf_model = RandomForestClassifier(max_features="auto", n_jobs=1,random_state=44)
rf_model = RandomForestClassifier(max_features="auto", n_jobs=1,random_state=44)
rf_model.fit(X_train, y_train)

RandomForestClassifier(n_jobs=1, random_state=44)

In [20]:
y_pred = rf_model.predict(X_test)
#y_test = buildMulticlassTestArrays(targets_set, y_test)

In [21]:
gc.collect()

48

In [22]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 4  0  0 ...  2  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0 13 ...  0  0  0]
 ...
 [ 0  0  0 ... 19  0  0]
 [ 0  0  0 ...  0  7  0]
 [ 0  0  0 ...  0  0  1]]


C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                     precision    recall  f1-score   support

               .net       0.80      0.03      0.06       127
               ajax       0.00      0.00      0.00         4
          algorithm       0.76      0.25      0.38        51
amazon-web-services       0.73      0.29      0.41        28
            android       0.79      0.88      0.83       801
     android-studio       1.00      0.12      0.22        16
            angular       0.67      0.59      0.63       116
          angularjs       0.77      0.15      0.26        65
             arrays       1.00      0.03      0.06        31
            asp.net       0.17      0.01      0.03        67
        asp.net-mvc       0.44      0.08      0.13        51
               bash       0.65      0.71      0.68       138
                  c       0.66      0.25      0.37       194
                 c#       0.58      0.75      0.65      1141
                c++       0.74      0.69      0.71       645
              c++11    

C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
# rf_model.base_estimator_
# rf_model.estimators_
len(rf_model.estimators_)

100

In [24]:
# import pickle
# s = pickle.dumps(rf_model)
# rf_model_2 = pickle.loads(s)

In [25]:
from joblib import dump, load
dump(rf_model, 'models/rf_model_less_tags.joblib') 

['models/rf_model_less_tags.joblib']

In [26]:
from sklearn.model_selection import cross_val_score
import numpy as np

np.mean(cross_val_score(rf_model, X_train, y_train, cv=5))

C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


0.6387879609998033